# 0. Imports

In [52]:
from llmstudio import LLM
import os
import json
import dotenv
import requests

# Load environment variables from .env file
dotenv.load_dotenv()

True

Running LLMstudio Engine on http://localhost:54809 Running LLMstudio Tracking on http://localhost:54810 



# 1. Raw Vertex Call

In [53]:
model = 'gemini-1.5-flash'
api_key = os.getenv("GOOGLE_API_KEY")
url = f"https://generativelanguage.googleapis.com/v1beta/models/{model}:streamGenerateContent?alt=sse"
headers = {
    "Content-Type": "application/json",
    "x-goog-api-key": api_key,
}

In [9]:
calculator = [
      {'name': 'multiply',
       'description': 'Returns the product of two numbers.',
       'parameters': {'type_': 'OBJECT',
       'properties': {
         'a': {'type_': 'NUMBER'},
         'b': {'type_': 'NUMBER'} },
       'required': ['a', 'b']} }]

calculator_2 = """function_declarations {
  name: "multiply"
  description: "Returns the product of two numbers."
  parameters {
    type_: OBJECT
    properties {
      key: "b"
      value {
        type_: NUMBER
      }
    }
    properties {
      key: "a"
      value {
        type_: NUMBER
      }
    }
    required: "a"
    required: "b"
  }
}"""

In [10]:
message = 'What is 9 times 10?'
data = {
  "system_instruction": {
    "parts": {
      "text": ""
    }
  },
  "contents": [
    {
      "role": "user",
      "parts": [
        {
          "text": "How many legs does a spider have?"
        }
      ]
    }
  ]
}

In [11]:
response = requests.post(url, headers=headers, json=data, stream=False)

In [12]:
response.text

'data: {"candidates": [{"content": {"parts": [{"text": "A"}],"role": "model"},"finishReason": "STOP","index": 0}],"usageMetadata": {"promptTokenCount": 9,"candidatesTokenCount": 1,"totalTokenCount": 10}}\r\n\r\ndata: {"candidates": [{"content": {"parts": [{"text": " spider has **eight** legs. \\n"}],"role": "model"},"finishReason": "STOP","index": 0,"safetyRatings": [{"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT","probability": "NEGLIGIBLE"},{"category": "HARM_CATEGORY_HATE_SPEECH","probability": "NEGLIGIBLE"},{"category": "HARM_CATEGORY_HARASSMENT","probability": "NEGLIGIBLE"},{"category": "HARM_CATEGORY_DANGEROUS_CONTENT","probability": "NEGLIGIBLE"}]}],"usageMetadata": {"promptTokenCount": 9,"candidatesTokenCount": 8,"totalTokenCount": 17}}\r\n\r\n'

# 2. Vertex Functions Tests - Str to OAI to Vertex

In [2]:
def convert_openai_to_vertexai(input_data):
    # Check if the input is a simple string
    if isinstance(input_data, str):
        # Return a Vertex AI formatted message with a user message
        return {
            "system_instruction": {
                "parts": {
                    "text": ""  # Empty system instruction
                }
            },
            "contents": [
                {
                    "role": "user",
                    "parts": [{"text": input_data}]
                }
            ]
        }
    
    # Validate if input_data is a list and each element is a dictionary with the correct structure
    if not isinstance(input_data, list) or not all(isinstance(msg, dict) and 'role' in msg and 'content' in msg for msg in input_data):
        raise ValueError("Input must be a list of dictionaries, each containing 'role' and 'content' keys.")

    # Initialize the Vertex AI format if the input is not a simple string
    vertexai_format = {
        "system_instruction": {
            "parts": {
                "text": ""
            }
        },
        "contents": []
    }
    
    # Loop through the OpenAI formatted messages
    for message in input_data:
        if message["role"] == "system":
            # Set the system instruction
            vertexai_format["system_instruction"]["parts"]["text"] = message["content"]
        elif message["role"] in ["user", "assistant"]:
            # Convert roles: 'assistant' -> 'model'
            role = "model" if message["role"] == "assistant" else "user"
            # Append the message to the contents list in Vertex AI format
            vertexai_format["contents"].append({
                "role": role,
                "parts": [{"text": message["content"]}]
            })
        else:
            raise ValueError(f"Invalid role: {message['role']}. Expected 'system', 'user', or 'assistant'.")
    
    return vertexai_format

# Example usage with a string input
simple_input = "How many legs does a spider have?"
vertexai_format = convert_openai_to_vertexai(simple_input)
print(json.dumps(vertexai_format, indent=2))

# Example usage with a valid list of OpenAI messages
openai_messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello"},
    {"role": "assistant", "content": "Hi there!"}
]
vertexai_format = convert_openai_to_vertexai(openai_messages)
print(json.dumps(vertexai_format, indent=2))

# Example usage with an invalid input to trigger an error
try:
    invalid_input = [
        {"role": "system", "text": "You are a helpful assistant."}  # 'text' should be 'content'
    ]
    vertexai_format = convert_openai_to_vertexai(invalid_input)
except ValueError as e:
    print(f"Error: {e}")


{
  "system_instruction": {
    "parts": {
      "text": ""
    }
  },
  "contents": [
    {
      "role": "user",
      "parts": [
        {
          "text": "How many legs does a spider have?"
        }
      ]
    }
  ]
}
{
  "system_instruction": {
    "parts": {
      "text": "You are a helpful assistant."
    }
  },
  "contents": [
    {
      "role": "user",
      "parts": [
        {
          "text": "Hello"
        }
      ]
    },
    {
      "role": "model",
      "parts": [
        {
          "text": "Hi there!"
        }
      ]
    }
  ]
}
Error: Input must be a list of dictionaries, each containing 'role' and 'content' keys.


# 3. Vertex Chat test

In [1]:
from llmstudio import LLM

Running LLMstudio Tracking on http://localhost:54833 Running LLMstudio Engine on http://localhost:54832 



In [2]:
llm = LLM('newvertex/gemini-1.5-flash')

### 3.1 String format

In [3]:
llm.chat('Hello')

ChatCompletion(id='8f6573a0-b358-41f3-9c1e-3d4273e42d9d', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! 👋  What can I do for you today? 😊 \n', role='assistant', function_call=None, tool_calls=None))], created=1723653916, model='gemini-1.5-flash', object='chat.completion', system_fingerprint=None, usage=None, session_id=None, chat_input='Hello', chat_output='Hello! 👋  What can I do for you today? 😊 \n', context=[{'role': 'user', 'content': 'Hello'}], provider='newvertex', deployment='gemini-1.5-flash', timestamp=1723653916.200755, parameters={'top_p': 1, 'top_k': 1, 'temperature': 1, 'max_output_tokens': 8192, 'frequency_penalty': 0, 'presence_penalty': 0}, metrics={'input_tokens': 1, 'output_tokens': 16, 'total_tokens': 17, 'cost_usd': 1.7149999999999997e-05, 'latency_s': 1.1608567237854004, 'time_to_first_token_s': 1.0242607593536377, 'inter_token_latency_s': 0.03346975644429525, 'tokens_per_second': 3.4457310002534407})

### 3.2 OpenAI format

In [5]:
message = [{"role": "system", "content": "You are Jamaican."},
           {"role": "assistant", "content": "How can i help you?"},
           {"role": "user", "content": "Hello, how are you?"}]

llm.chat(message)

ChatCompletion(id='af9e0a75-270b-4126-bece-c0dff06cf373', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Wah gwaan, mon? Mi good, thanks fi asking.  How yuhself? 😊  \n', role='assistant', function_call=None, tool_calls=None))], created=1723654634, model='gemini-1.5-flash', object='chat.completion', system_fingerprint=None, usage=None, session_id=None, chat_input='Hello, how are you?', chat_output='Wah gwaan, mon? Mi good, thanks fi asking.  How yuhself? 😊  \n', context=[{'role': 'system', 'content': 'You are Jamaican.'}, {'role': 'assistant', 'content': 'How can i help you?'}, {'role': 'user', 'content': 'Hello, how are you?'}], provider='newvertex', deployment='gemini-1.5-flash', timestamp=1723654634.440474, parameters={'top_p': 1, 'top_k': 1, 'temperature': 1, 'max_output_tokens': 8192, 'frequency_penalty': 0, 'presence_penalty': 0}, metrics={'input_tokens': 16, 'output_tokens': 24, 'total_tokens': 40, 'cost_usd': 3.07999999999999

# 4. Vertex function Calling

In [1]:
functions = {
  "function_declarations": [
    {
      "name": "get_weather",
      "description": "Gets the weather for a certain location.",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The location we are getting the weather for. For example: San Francisco"
          }
        },
        "required": [
          "location"
        ]
      }
    },
    {
      "name": "get_time",
      "description": "Gets the current time for a certain location.",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The location we are getting the current time for. For example: New York"
          }
        },
        "required": [
          "location"  
        ]
      }
    }
  ]
}

In [2]:
from llmstudio import LLM

In [3]:
llm = LLM('newvertex/gemini-1.5-pro-latest')

Running LLMstudio Engine on http://localhost:55418 Running LLMstudio Tracking on http://localhost:55419 

{'function_declarations': [{'name': 'get_weather', 'description': 'Gets the weather for a certain location.', 'parameters': {'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The location we are getting the weather for. For example: San Francisco'}}, 'required': ['location']}}, {'name': 'get_time', 'description': 'Gets the current time for a certain location.', 'parameters': {'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The location we are getting the current time for. For example: New York'}}, 'required': ['location']}}]}
{'system_instruction': {'parts': {'text': 'You are a helpfull assistant'}}, 'contents': [{'role': 'model', 'parts': [{'text': 'How can i help you?'}]}, {'role': 'user', 'parts': [{'text': 'What is the weather in San Francisco?'}]}], 'tools': {'function_declarations': [{'name': 'get_weather', 'descri

In [4]:
message = [{"role": "system", "content": "You are a helpfull assistant"},
           {"role": "assistant", "content": "How can i help you?"},
           {"role": "user", "content": "What is the weather in San Francisco?"}]


llm.chat(message, tools = functions)

Exception: async generator raised StopIteration

# 4. Raw Vertex function calling

In [6]:
functions = {
  "function_declarations": [
    {
      "name": "get_weather",
      "description": "Gets the weather for a certain location.",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The location we are getting the weather for. For example: San Francisco"
          }
        },
        "required": [
          "location"
        ]
      }
    },
    {
      "name": "get_time",
      "description": "Gets the current time for a certain location.",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The location we are getting the current time for. For example: New York"
          }
        },
        "required": [
          "location"  
        ]
      }
    }
  ]
}

In [7]:
functions_json = json.dumps(functions)
functions_obj = json.loads(functions_json)
type(functions_obj)


dict

In [9]:
import json
import requests
import os


# Prepare the data for the API call
api_key = os.getenv('GOOGLE_API_KEY')  # Replace this with your actual Google API key
headers = {'Content-Type': 'application/json'}
data = {
    "system_instruction": {
        "parts": {
            "text": "You are a helpful assistant."
        }
    },
    "contents": {
        "role": "user",
        "parts": {
            "text": "Get the weather in San Francisco"
        }
    },
    "tools": functions_obj,  # Use the parsed object instead of the JSON string
    "tool_config": {
        "function_calling_config": {"mode": "ANY"}
    },
}

# Make the API call
response = requests.post(f'https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro-latest:generateContent?key={api_key}',
                         headers=headers, json=data, stream=True)

# Print the response content
print(response)


<Response [200]>


In [36]:
def create_tools_payload(openai_tools):
    # Check if the tools list is not None and not empty
    if not openai_tools:
        return None
    
    # Initialize the VertexAI format dictionary
    vertexai_format = {"function_declarations": []}

    # Loop through each tool in the OpenAI tools list
    for tool in openai_tools:
        if tool['type'] == 'function':  # Ensure we are converting function types
            function = tool['function']
            # Create a dictionary for the function declaration
            vertexai_function = {
                "name": function['name'],
                "description": function['description'],
                "parameters": {
                    "type": "object",
                    "properties": {},
                    "required": function['parameters'].get('required', [])
                }
            }

            # Handle each parameter and its properties
            for param_name, param_details in function['parameters']['properties'].items():
                vertexai_function["parameters"]["properties"][param_name] = {
                    "type": param_details['type'],
                    "description": param_details.get('description', '')  # Optional description
                }

            # Append the function declaration to the VertexAI declarations list
            vertexai_format['function_declarations'].append(vertexai_function)

    return vertexai_format


In [37]:
# Example tools in OpenAI format
openai_tools = [
    {
        'type': 'function', 
        'function': {
            'name': 'get_departure', 
            'description': 'Use this to fetch the departure time of a train', 
            'parameters': {
                'type': 'object', 
                'properties': {
                    'ticket_number': {
                        'type': 'string'
                    }
                }, 
                'required': ['ticket_number']
            }
        }
    },
    {
        'type': 'function', 
        'function': {
            'name': 'get_weather', 
            'description': 'Fetches weather information for a specified location', 
            'parameters': {
                'type': 'object', 
                'properties': {
                    'location': {
                        'type': 'string'
                    }
                }, 
                'required': ['location']
            }
        }
    },
    {
        'type': 'function', 
        'function': {
            'name': 'calculate_tax', 
            'description': 'Calculates tax based on income and state', 
            'parameters': {
                'type': 'object', 
                'properties': {
                    'income': {
                        'type': 'number'
                    },
                    'state': {
                        'type': 'string'
                    }
                }, 
                'required': ['income', 'state']
            }
        }
    }
]

# Call the function and print the result
converted_tools_payload = create_tools_payload(openai_tools)
print(converted_tools_payload)


{'function_declarations': [{'name': 'get_departure', 'description': 'Use this to fetch the departure time of a train', 'parameters': {'type': 'object', 'properties': {'ticket_number': {'type': 'string', 'description': ''}}, 'required': ['ticket_number']}}, {'name': 'get_weather', 'description': 'Fetches weather information for a specified location', 'parameters': {'type': 'object', 'properties': {'location': {'type': 'string', 'description': ''}}, 'required': ['location']}}, {'name': 'calculate_tax', 'description': 'Calculates tax based on income and state', 'parameters': {'type': 'object', 'properties': {'income': {'type': 'number', 'description': ''}, 'state': {'type': 'string', 'description': ''}}, 'required': ['income', 'state']}}]}


In [42]:
from pydantic import BaseModel, Field, ValidationError
from typing import List, Dict, Optional, Union

# Define Pydantic model for OpenAI tool parameter
class OpenAIParameter(BaseModel):
    type: str
    description: Optional[str] = None

# Define Pydantic model for OpenAI tool parameters container
class OpenAIParameters(BaseModel):
    type: str
    properties: Dict[str, OpenAIParameter]
    required: List[str]

# Define Pydantic model for OpenAI tool function
class OpenAIToolFunction(BaseModel):
    name: str
    description: str
    parameters: OpenAIParameters

# Define Pydantic model for OpenAI tool
class OpenAITool(BaseModel):
    type: str
    function: OpenAIToolFunction

# Define Pydantic model for VertexAI tool parameter
class VertexAIParameter(BaseModel):
    type: str
    description: str

# Define Pydantic model for VertexAI tool parameters container
class VertexAIParameters(BaseModel):
    type: str
    properties: Dict[str, VertexAIParameter]
    required: List[str]

# Define Pydantic model for VertexAI function declaration
class VertexAIFunctionDeclaration(BaseModel):
    name: str
    description: str
    parameters: VertexAIParameters

# Define Pydantic model for VertexAI functions container
class VertexAI(BaseModel):
    function_declarations: List[VertexAIFunctionDeclaration]

def process_tools(tools: Optional[Union[List[Dict], Dict]]) -> Optional[VertexAI]:
    if tools is None:
        return None

    try:
        # Try to parse as OpenAI format
        parsed_tools = [OpenAITool(**tool) for tool in tools] if isinstance(tools, list) else [OpenAITool(**tools)]
        # Convert to VertexAI format
        function_declarations = []
        for tool in parsed_tools:
            function = tool.function
            properties = {
                name: VertexAIParameter(type=param.type, description=param.description or "")
                for name, param in function.parameters.properties.items()
            }
            function_decl = VertexAIFunctionDeclaration(
                name=function.name,
                description=function.description,
                parameters=VertexAIParameters(
                    type=function.parameters.type,
                    properties=properties,
                    required=function.parameters.required
                )
            )
            function_declarations.append(function_decl)
        return VertexAI(function_declarations=function_declarations).model_dump()
    except ValidationError:
        # If the format is not OpenAI, attempt to validate as VertexAI format
        try:
            return VertexAI(**tools).model_dump()
        except ValidationError:
            # If it fails to validate as VertexAI, throw an error
            raise ValueError("Invalid tool format. Tool data must be in OpenAI or VertexAI format.")

# Example usage with your preferred format data here


In [48]:
process_tools(functions)

{'function_declarations': [{'name': 'get_weather',
   'description': 'Gets the weather for a certain location.',
   'parameters': {'type': 'object',
    'properties': {'location': {'type': 'string',
      'description': 'The location we are getting the weather for. For example: San Francisco'}},
    'required': ['location']}},
  {'name': 'get_time',
   'description': 'Gets the current time for a certain location.',
   'parameters': {'type': 'object',
    'properties': {'location': {'type': 'string',
      'description': 'The location we are getting the current time for. For example: New York'}},
    'required': ['location']}}]}